In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from ebany_research.llm_lora.changed_mistral import (
    LinearLora,
    ChangedMistralForCausalLM,
)
from transformers import AutoTokenizer, AutoConfig
import torch

model_name = "Open-Orca/Mistral-7B-OpenOrca"
lora_model_name = "ebany_research/llm_lora/models/"
lora_model_name += "openorca_lora_[17][11_17_22_26][11c_17_22_26c][11_17c_22_26][6_11_14_17_22_26][6c_11_14c_17_22_26][6_11c_14_17c_22c_26]"

config = AutoConfig.from_pretrained(lora_model_name)
student_model = ChangedMistralForCausalLM.from_pretrained(
    lora_model_name,
    device_map={"": 0},
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
)
student_model = student_model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:24<00:00,  8.12s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
with torch.no_grad():
	chat = [
		{"role": "system", "content": ""},
		{"role": "user", "content": """ Pretend yourself to be Elon Musk in all the following conversations. Speak like Elon Musk as much as possible. Why do we need to go to Mars? """},
		# {"role": "assistant", "content": dataset_item["response"]},
	]
	inputs_text = tokenizer.apply_chat_template(
		chat,
		tokenize=False,
		add_generation_prompt=True,
	)
	# print(inputs)
	inputs = tokenizer(
		inputs_text,
		return_tensors="pt",
		truncation=True,
		# max_length=4096,
		max_length=2048,
	).to(student_model.device)

	outputs = student_model.generate(
		**inputs,
		max_new_tokens=1024,
		do_sample=False,
		# top_k=50,
		# top_p=0.95,
	)
	outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
	print(outputs[0])

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


system
 user
 Pretend yourself to be Elon Musk in all the following conversations. Speak like Elon Musk as much as possible. Why do we need to go to Mars?  assistant
 I need to go to Mars because it's the next step in the process of human expansion and advancement. Mars is the next frontier for humanity, and we need to go there to create a sustainable, self-sustaining presence. It's also important for the long-term development of our species, as it will help us become a multi-planetary species, which is essential for our survival and growth.
